# Writing Programs for Aquila

## Query Aquila capabilities 
To see the docstring in a Jupyter notebook, type out the object you're interested in and press `shift + tab`.

The `bloqade function` `get_capabilities` returns the parameter value limits for Aquillia. 

We can use `shift + tab` to see its docstring, which defines the units for the returned values.

In [2]:
import pprint # A standard Python library for 'pretty printing'
from bloqade import get_capabilities 

capabilities = get_capabilities().dict()['capabilities']
pprint.pprint(capabilities)

{'lattice': {'area': {'height': Decimal('76'), 'width': Decimal('75')},
             'geometry': {'number_sites_max': 256,
                          'position_resolution': Decimal('0.1'),
                          'spacing_radial_min': Decimal('4'),
                          'spacing_vertical_min': Decimal('4')},
             'number_qubits_max': 256},
 'rydberg': {'c6_coefficient': Decimal('5.42E+6'),
             'global_': {'detuning_max': Decimal('125.0000000'),
                         'detuning_min': Decimal('-125.0000000'),
                         'detuning_resolution': Decimal('2E-7'),
                         'detuning_slew_rate_max': Decimal('2500.0000000000000'),
                         'phase_max': Decimal('99.0'),
                         'phase_min': Decimal('-99.0'),
                         'phase_resolution': Decimal('5E-7'),
                         'rabi_frequency_max': Decimal('15.8000000'),
                         'rabi_frequency_min': Decimal('0E-7'),
         

In [5]:
# this cell is an example of what you would see with shift + tab
# great way to check the units 
get_capabilities

<function bloqade.factory.get_capabilities(use_experimental: bool = False) -> 'QuEraCapabilities'>

## Running some bloqade geometries

To write a program for Aquila, we start with an atom geometry. Now lets try running some setups. Let's create a Chain of 4 atoms that are each $50 \, \mu m$ apart. Take a look at its methods and attributes.

**Note**: Geometry is actually important on the real system because certain geometries are less noise than others (for example vertical rather than horizontal.)

*Remember*: to see the methods and attributes of an object instance, first instantiate the object by evaluating the cell (`shift + return`), then type `.` followed by `tab`.


In [6]:
from math import pi
from bloqade.atom_arrangement import Chain

In [10]:
geometry = Chain(4, lattice_spacing = 50.0)
# you can always look at the geometry visually.
geometry

                                   Atom Positions                               
     ┌─────────────────────────────────────────────────────────────────────────┐
 1.00┤                                                                         │
     │                                                                         │
     │                                                                         │
 0.67┤                                                                         │
     │                                                                         │
     │                                                                         │
 0.33┤                                                                         │
     │                                                                         │
     │                                                                         │
 0.00┤•                       •                       •                       •│
     │                      

Methods to create pulse sequences that manipulate the rabi pulse or local detunings are found under `geometry.rydberg`.

In [11]:
# Again, try a shift+tab here
geometry.rydberg

## Evolving the system 

Let's create a simple program that applies a NOT gate to the initial $\ket{1111}$ state, by applying a $\pi$ Rabi pulse (a $\pi$ rotation on the Bloch sphere). 

*Remember*: 
* Atoms are initialised in the $\ket{1}$ state. 
* All atoms (qubits) undergo this evolution so you should get $\ket{1111}\rightarrow\ket{0000}$. 

In [12]:
NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .constant(pi,1)
)

We can simulate the program with the Python backend as follows.

**NOTE**: $10000$ shots would cost USD $~1300$! (Also, is it even within the device capability limits?)

In [13]:
n_shots = 10000
results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 10000)])]

The `results` object contains quite a bit of information, use `results.report().show()` to open a visualisation of the simulation results.
**Note** that the above results are for an *ideal* system and setup. 

In [18]:
# this can be a bit temperamental and also will open a window in the default browser
results.report().show()

0.0 150000000.0 0.0 0.0


To run a simulation on Aquilla the atom geometry and pulse sequence must be within the its capability limits.

With that in mind, there are a couple issues with our `NOT` program:

* The chain has a horizontal length of $200 \, \mu m$, while the maximum width is $75 \, \mu m$.
* A `constant` pulse is not possible in practice; the Rabi amplitude waveform must start and end at zero.

In [19]:
# Reducing the atom spacing.

geometry = Chain(4, lattice_spacing = 15.0)

NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .constant(pi,1)
)

results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 9302),
              ('0100', 247),
              ('0010', 231),
              ('1000', 64),
              ('0001', 47),
              ('0110', 45),
              ('1100', 29),
              ('0011', 28),
              ('1010', 3),
              ('0101', 2),
              ('1101', 1),
              ('1110', 1)])]

Now instead the results show a large number of *incorrect* answers but the correct result accounts for $93\%$. Yet this is not representative of what could be run on the system. We need to adjust the pulse. Instead of `uniform` we need to specify a continuous pulse sequence. Here we use `peicewise_linear`.

In [20]:
NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .piecewise_linear(values = [0, pi, pi, 0], durations = [0.06, 1, 0.06])
)

results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 8972),
              ('0100', 287),
              ('0010', 262),
              ('0001', 140),
              ('1000', 115),
              ('0110', 87),
              ('0011', 62),
              ('1100', 60),
              ('0101', 4),
              ('1010', 3),
              ('1011', 3),
              ('1110', 3),
              ('0111', 2)])]

Again, the correct result is $\less100\%$ and has even gotten worse. 

As the pulse now has a ramp-up and ramp-dow period, we need to factor this into the total pulse area.

By reducing the time spent at a Rabi amplitude of $\pi$ based on the area of the ramp-up and ramp-down periods, the accuracy of the `NOT` program improves.

In [14]:
area_correction = 2*0.06*0.5

NOT = (
  geometry
  .rydberg.rabi.amplitude.uniform
  .piecewise_linear(values = [0, pi, pi, 0], durations = [0.06, 1 - area_correction, 0.06])
)

results = NOT.bloqade.python().run(n_shots)
results.report().counts()

[OrderedDict([('0000', 9317),
              ('0010', 231),
              ('0100', 220),
              ('1000', 66),
              ('0001', 56),
              ('0110', 42),
              ('1100', 34),
              ('0011', 25),
              ('1010', 4),
              ('0111', 2),
              ('0101', 1),
              ('1001', 1),
              ('1110', 1)])]

Now we are back to $93\%$. Yet this is for lots of shots AND also assumes an ideal system. There is no noise here. This is a key thing to consider. Now consider trying this using a job submission script and compare the results.

# CNOT?

In [1]:
from math import pi
from bloqade import start
from bloqade.atom_arrangement import Chain


pulse_sequence = (start.rydberg
    .detuning
    .location(0)
    .piecewise_linear([0.06, 1, 0.06, 0.06, 1, 0.06, 0.06, 1, 0.06],[0, -50, -50, 0, 0, 0, 0, -50, -50, 0]))


pulse_sequence = (pulse_sequence.rydberg
    .detuning
    .location(1)
    .piecewise_linear([1.12, 0.06, 1, 0.06],[0, 0, -50, -50, 0]))

pulse_sequence = (pulse_sequence
    .rydberg
    .rabi
    .amplitude
    .uniform
    .piecewise_linear([0.06, 1, 0.06, 0.06, 1, 0.06, 0.06, 1, 0.06],[0, pi, pi, 0, pi, pi, 0, pi, pi, 0]))

pulse_sequence = pulse_sequence.parse_sequence()

geometry = Chain(2, lattice_spacing = 6)

program = geometry.apply(pulse_sequence)

program.show()

results = program.bloqade.python().run(100)

bitstring_counts = results.report().counts()
print(bitstring_counts)


[OrderedDict({'11': 95, '10': 3, '01': 2})]
